In [1]:
from CANdb_functions import *


In [12]:
messages_and_signals = pd.read_csv("CSVs/RAW_MESSAGES.csv")
print(messages_and_signals.columns)


Index(['MSG ID', 'Message Name', 'Length', 'Byte', 'Bit Start', 'Data Name',
       'Description', 'Data Type', 'Min', 'Max', 'Scaled Min', 'Scaled Max',
       'Map Equation', 'Units', 'For Indexing Purposes', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19'],
      dtype='object')


In [13]:
messages = messages_and_signals[["MSG ID", "Message Name", "Length"]].dropna(subset=["Message Name"])
print(messages.info())
messages = messages


<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 0 to 948
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MSG ID        35 non-null     object 
 1   Message Name  35 non-null     object 
 2   Length        35 non-null     float64
dtypes: float64(1), object(2)
memory usage: 1.1+ KB
None


In [17]:
# funny solution because message sheets format is dog

CANdb = cantools.database.load_file(dbc_file)

nodes = [node.name for node in CANdb.nodes]

# Function to find a sender node by matching first 3 letters
def find_sender(message_name):
    prefix = message_name[:3].replace(" ", "_").lower()  
    matches = [n for n in nodes if prefix in n.lower()]   
    return matches[0] if matches else "ECU" 


messages["Sender"] = messages["Message Name"].apply(find_sender)

print(messages.head())
# messages.to_csv("CSVs/MESSAGES.csv", index=False)


   MSG ID Message Name  Length    Sender
0   0x000        Debug    64.0  Debugger
2   0x001         Ping     4.0       ECU
4   0x002   ECU status    44.0       ECU
41  0x003   ECU config     0.0       ECU
43  0x004   ACU Status    24.0       ACU
